# Detection of Fraud Reviews: Modeling and Evaluation

# Loading libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Reading datasets

## Train dataset

In [2]:
# Path to your CSV file
X_train_merged_path = '/Users/LeeShan/Fake-Reviews-Detection/ShanShan_notebooks/X_train_merged.csv'

y_train_merged_path = '/Users/LeeShan/Fake-Reviews-Detection/ShanShan_notebooks/y_train_merged.csv'

# Read the CSV file
X_train_merged = pd.read_csv(X_train_merged_path)

y_train_merged = pd.read_csv(y_train_merged_path)

Information

In [3]:
X_train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542931 entries, 0 to 542930
Columns: 114 entries, word_count to year
dtypes: float64(112), int64(2)
memory usage: 472.2 MB


In [4]:
y_train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542931 entries, 0 to 542930
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   label   542931 non-null  int64
dtypes: int64(1)
memory usage: 4.1 MB


Shape

In [5]:
X_train_merged.shape

(542931, 114)

In [6]:
y_train_merged.shape

(542931, 1)

Columns

In [7]:
X_train_merged.columns

Index(['word_count', 'avg_word_length', 'avg_sentence_length',
       'uppercase_char_count', 'rating', 'Sentiment_Neutral',
       'Sentiment_Positive', 'category_Clothing_Shoes_and_Jewelry_5',
       'category_Electronics_5', 'category_Food_5',
       ...
       'wa', 'wait', 'want', 'wanted', 'way', 'well', 'without', 'worth',
       'would', 'year'],
      dtype='object', length=114)

In [8]:
y_train_merged.columns

Index(['label'], dtype='object')

Structure

In [9]:
# Display the first few rows
X_train_merged.head()

,word_count,avg_word_length,avg_sentence_length,uppercase_char_count,rating,Sentiment_Neutral,Sentiment_Positive,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Food_5,...,wa,wait,want,wanted,way,well,without,worth,would,year
0,4,7.000000,4.000000,1,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,159,4.226415,17.666667,17,5.0,0.0,1.0,0.0,0.0,1.0,...,0.163465,0.0,0.000000,0.0,0.0,0.104663,0.0,0.0,0.175222,0.0
2,56,4.125000,18.666667,9,4.0,0.0,1.0,0.0,0.0,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,171,3.970760,9.500000,24,5.0,0.0,1.0,0.0,0.0,1.0,...,0.000000,0.0,0.326243,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,291,4.058419,13.857143,24,5.0,0.0,1.0,0.0,0.0,1.0,...,0.045875,0.0,0.000000,0.0,0.0,0.088118,0.0,0.0,0.000000,0.0


In [10]:
y_train_merged.head()

,label
0,0
1,0
2,0
3,0
4,0


## Test dataset

In [11]:
# Path to your CSV file
X_test_merged_path = '/Users/LeeShan/Fake-Reviews-Detection/ShanShan_notebooks/X_test_merged.csv'

y_test_merged_path = '/Users/LeeShan/Fake-Reviews-Detection/ShanShan_notebooks/y_test_merged.csv'

# Read the CSV file
X_test_merged = pd.read_csv(X_test_merged_path)

y_test_merged = pd.read_csv(y_test_merged_path)

Information

In [12]:
X_test_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79124 entries, 0 to 79123
Columns: 114 entries, word_count to year
dtypes: float64(112), int64(2)
memory usage: 68.8 MB


In [13]:
y_test_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79124 entries, 0 to 79123
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   79124 non-null  int64
dtypes: int64(1)
memory usage: 618.3 KB


Shape

In [14]:
X_test_merged.shape

(79124, 114)

In [15]:
y_test_merged.shape

(79124, 1)

Columns

In [16]:
X_test_merged.columns

Index(['word_count', 'avg_word_length', 'avg_sentence_length',
       'uppercase_char_count', 'rating', 'Sentiment_Neutral',
       'Sentiment_Positive', 'category_Clothing_Shoes_and_Jewelry_5',
       'category_Electronics_5', 'category_Food_5',
       ...
       'wa', 'wait', 'want', 'wanted', 'way', 'well', 'without', 'worth',
       'would', 'year'],
      dtype='object', length=114)

In [17]:
y_test_merged.columns

Index(['label'], dtype='object')

Structure

In [18]:
# Display the first few rows
X_test_merged.head()

,word_count,avg_word_length,avg_sentence_length,uppercase_char_count,rating,Sentiment_Neutral,Sentiment_Positive,category_Clothing_Shoes_and_Jewelry_5,category_Electronics_5,category_Food_5,...,wa,wait,want,wanted,way,well,without,worth,would,year
0,161,4.279503,14.636364,16,4.0,0.0,1.0,0.0,0.0,1.0,...,0.055828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,38,4.394737,12.666667,4,5.0,0.0,1.0,0.0,0.0,1.0,...,0.140718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,93,3.892473,13.285714,27,5.0,0.0,1.0,0.0,0.0,1.0,...,0.255208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.229119
3,61,4.147541,7.625000,11,4.0,0.0,1.0,0.0,0.0,1.0,...,0.071309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114656,0.000000
4,68,3.897059,11.333333,6,4.0,0.0,1.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157213,0.000000


In [19]:
# Display the first few rows
y_test_merged.head()

,label
0,0
1,0
2,0
3,0
4,0


# Check for missing values

## Train dataset

In [20]:
# Check for missing values
X_train_merged.isnull().sum()

word_count              0
avg_word_length         0
avg_sentence_length     0
uppercase_char_count    0
rating                  0
                       ..
well                    0
without                 0
worth                   0
would                   0
year                    0
Length: 114, dtype: int64

In [21]:
# Check for missing values
y_train_merged.isnull().sum()

label    0
dtype: int64

## Test dataset

In [22]:
# Check for missing values
X_test_merged.isnull().sum()

word_count              0
avg_word_length         0
avg_sentence_length     0
uppercase_char_count    0
rating                  0
                       ..
well                    0
without                 0
worth                   0
would                   0
year                    0
Length: 114, dtype: int64

In [23]:
# Check for missing values
y_test_merged.isnull().sum()

label    0
dtype: int64

# Modeling

## Naive Bayes Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, roc_curve, RocCurveDisplay
)
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier

# Placeholder for training and testing datasets
# Replace these with your actual datasets
X_train_merged = np.random.rand(1000, 20)  # Dummy data for illustration
y_train_merged = np.random.randint(0, 2, 1000)
X_test_merged = np.random.rand(300, 20)
y_test_merged = np.random.randint(0, 2, 300)

# Define models with default parameters
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Random Forest": RandomForestClassifier()
}

# DataFrames to store performance metrics
results = []

# Loop through models to train, evaluate, and visualize
for model_name, model in models.items():
    print(f"Training and evaluating: {model_name}")

    # Duplicate datasets for reuse
    X_train_dup = X_train_merged.copy()
    y_train_dup = y_train_merged.copy()
    X_test_dup = X_test_merged.copy()
    y_test_dup = y_test_merged.copy()
    
    # Fit the model
    model.fit(X_train_dup, y_train_dup)
    
    # Predict on test set
    y_pred = model.predict(X_test_dup)
    y_prob = model.predict_proba(X_test_dup)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test_dup, y_pred)
    precision = precision_score(y_test_dup, y_pred)
    recall = recall_score(y_test_dup, y_pred)
    f1 = f1_score(y_test_dup, y_pred)
    auc_score = roc_auc_score(y_test_dup, y_prob)
    report = classification_report(y_test_dup, y_pred)
    cm = confusion_matrix(y_test_dup, y_pred)
    
    # Save metrics
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "AUC": auc_score,
        "Classification Report": report,
        "Confusion Matrix": cm
    })
    
    # Print classification report
    print(f"\nClassification Report for {model_name}:\n")
    print(report)
    
    # Plot confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
                xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.title(f"Confusion Matrix for {model_name}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

# Plot ROC-AUC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
for idx, (model_name, model) in enumerate(models.items()):
    y_prob = model.predict_proba(X_test_merged)[:, 1]
    fpr, tpr, _ = roc_curve(y_test_merged, y_prob)
    auc_score = roc_auc_score(y_test_merged, y_prob)
    
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc_score, estimator_name=model_name).plot(ax=plt.gca(), 
                                                                                         color=colors[idx], 
                                                                                         label=f"{model_name} (AUC={auc_score:.2f})")

# Customize the plot
plt.title("ROC-AUC Curves for Models")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

# Display results as a DataFrame
results_df = pd.DataFrame(results)
print("\nOverall Results Summary:")
print(results_df[["Model", "Accuracy", "Precision", "Recall", "F1-Score", "AUC"]])

# --------------------------------------------------------------------
# Hyperparameter Tuning Section

# Define hyperparameter grids for each model
param_grids = {
    "Naive Bayes": {},  # No hyperparameters for Naive Bayes in this context
    "Logistic Regression": {
        "solver": ["newton-cg", "lbfgs", "liblinear"],
        "C": np.logspace(-4, 4, 20),
        "max_iter": [100, 200, 500]
    },
    "Gradient Boosting": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7],
    },
    "AdaBoost": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.5, 1.0, 1.5]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    }
}

# Perform Randomized Search and evaluate
def run_with_tuning(models, param_grids):
    tuned_results = []
    for model_name, model in models.items():
        print(f"\nTuning and evaluating: {model_name}")
        
        # RandomizedSearchCV for hyperparameter tuning
        if param_grids[model_name]:
            search = RandomizedSearchCV(
                estimator=model,
                param_distributions=param_grids[model_name],
                n_iter=20,
                scoring='roc_auc',
                cv=3,
                verbose=1,
                random_state=42
            )
            search.fit(X_train_merged, y_train_merged)
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
        else:
            best_model = model
            best_model.fit(X_train_merged, y_train_merged)
        
        # Predictions and metrics
        y_pred = best_model.predict(X_test_merged)
        y_prob = best_model.predict_proba(X_test_merged)[:, 1]
        
        accuracy = accuracy_score(y_test_merged, y_pred)
        precision = precision_score(y_test_merged, y_pred)
        recall = recall_score(y_test_merged, y_pred)
        f1 = f1_score(y_test_merged, y_pred)
        auc_score = roc_auc_score(y_test_merged, y_prob)
        report = classification_report(y_test_merged, y_pred)
        cm = confusion_matrix(y_test_merged, y_pred)
        
        # Save results
        tuned_results.append({
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "AUC": auc_score,
            "Classification Report": report,
            "Confusion Matrix": cm
        })
        
        # Print classification report
        print(f"\nClassification Report for {model_name}:\n")
        print(report)
        
        # Plot confusion matrix
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
                    xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
        plt.title(f"Confusion Matrix for {model_name}")
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.show()
    
    # Plot ROC-AUC curves
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
    for idx, result in enumerate(tuned_results):
        fpr, tpr, _ = roc_curve(y_test_merged, models[result["Model"]].predict_proba(X_test_merged)[:, 1])
        RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=result["AUC"], estimator_name=result["Model"]).plot(ax=plt.gca(),
                                                                                                    color=colors[idx],
                                                                                                    label=f"{result['Model']} (AUC={result['AUC']:.2f})")
    
    # Customize ROC plot
    plt.title("ROC-AUC Curves for Models (After Tuning)")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.grid(alpha=


# Logistic Regression

# Adaboost Model

# Gradient Boosting

# XGBoost

# Random Forest

In [ ]:
# TF-IDF: Naive Bayes, Logistic Regression, Gradient Boosting, Adaboost, XGBoost, Random Forest

# word2vec: Logistic Regression, Gradient Boosting, Adaboost, XGBoost, Random Forest

# Tokenization: Bert, Roberta, LSTM

# Fine-tune: RandomizedSearch


# tf-idf, word2vec